In [1]:

#import csv file to google collab
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data (2).csv


## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"],axis='columns')
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
ap_counts = application_df['APPLICATION_TYPE'].value_counts()
ap_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(ap_counts[ap_counts < 600].index)


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1728        1
C6100        1
C1283        1
C4120        1
C2600        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts > 10]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
ap_dummies_df = pd.get_dummies(application_df)

In [11]:
# Split our preprocessed data into our features and target arrays
X = ap_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = ap_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

# ** 1st Run Attempt**

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
no_input_features = len(X_train[0])
hidden_nodes_1 = 99
hidden_nodes_2 = 50
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_1, input_dim=no_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_2, input_dim=no_input_features, activation="sigmoid")
)


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 99)                4356      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5000      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 9,407
Trainable params: 9,407
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5914 - accuracy: 0.7090
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5727 - accuracy: 0.7234
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5671 - accuracy: 0.7243
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5621 - accuracy: 0.7251
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5580 - accuracy: 0.7274
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5559 - accuracy: 0.7269
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7271
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5534 - accuracy: 0.7283
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5520 - accuracy: 0.7295
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5510 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5548 - accuracy: 0.7282
Loss: 0.5548303723335266, Accuracy: 0.7281632423400879


# ** 2nd Run Attempt**

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
no_input_features = len(X_train[0])
hidden_nodes_1 = 50
hidden_nodes_2 = 25
hidden_nodes_3 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_1, input_dim=no_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_2, input_dim=no_input_features, activation="sigmoid")
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_3, input_dim=no_input_features, activation="sigmoid")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 50)                2200      
_________________________________________________________________
dense_4 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                260       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 3,746
Trainable params: 3,746
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6050 - accuracy: 0.6918
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5750 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5723 - accuracy: 0.7270
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5691 - accuracy: 0.7277
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5643 - accuracy: 0.7283
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5603 - accuracy: 0.7285
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5573 - accuracy: 0.7286
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7304
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7314
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5533 - accura

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5522 - accuracy: 0.7308
Loss: 0.5522181987762451, Accuracy: 0.7308454513549805


# ** 3rd Run Attempt **

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
no_input_features = len(X_train[0])
hidden_nodes_1 = 50
hidden_nodes_2 = 25
hidden_nodes_3 = 10
hidden_nodes_4 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_1, input_dim=no_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_2, input_dim=no_input_features, activation="sigmoid")
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_3, input_dim=no_input_features, activation="sigmoid")
)

# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_4, input_dim=no_input_features, activation="sigmoid")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 50)                2200      
_________________________________________________________________
dense_8 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                260       
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 6         
Total params: 3,796
Trainable params: 3,796
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6272 - accuracy: 0.6747
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5781 - accuracy: 0.7288
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5750 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5738 - accuracy: 0.7293
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5728 - accuracy: 0.7311
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5724 - accuracy: 0.7314
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5721 - accuracy: 0.7307
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5715 - accuracy: 0.7313
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5710 - accuracy: 0.7311
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5704 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5533 - accuracy: 0.7289
Loss: 0.5533269643783569, Accuracy: 0.728863000869751


# ** 4th Run Attempt **

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
no_input_features = len(X_train[0])
hidden_nodes_1 = 50
hidden_nodes_2 = 25
hidden_nodes_3 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_1, input_dim=no_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_2, input_dim=no_input_features, activation="sigmoid")
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_3, input_dim=no_input_features, activation="sigmoid")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 50)                2200      
_________________________________________________________________
dense_13 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_14 (Dense)             (None, 10)                260       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 3,746
Trainable params: 3,746
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6043 - accuracy: 0.6974
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5747 - accuracy: 0.7257
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5714 - accuracy: 0.7263
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5665 - accuracy: 0.7259
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5616 - accuracy: 0.7262
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5581 - accuracy: 0.7287
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5559 - accuracy: 0.7289
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5544 - accuracy: 0.7301
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7311
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5523 - accura

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5549 - accuracy: 0.7312
Loss: 0.5548874139785767, Accuracy: 0.731195330619812


### ** ANALYSIS: REPORT ON THE NEURAL NETWORK MODEL **

The target variable for the model will be "IS_SUCCESSFUL". This column has values of 1 and 0 which helps us determine if the charity fund is successful (1) or not successful (0). After columns "EIN" and "Name" are dropped, the remaining columns are features for the model.

**TASK**: Compile, train, and evaluate the model using TensorFlow to design a neural network/deep learning model, and create a binary classification model that can predict if the organization will be successful based on the features in the dataset. Compile, train, and evaluate the binary classification model to calculate the model’s loss and accuracy.

**Run 1** : Use neural network model with 2 hidden layers. 1st layer with 99 nodes and second layer with 50. Using 100 epochs.
 This model resulted in Loss: 0.5548303723335266, Accuracy: 0.7281632423400879.

**Run 2**: Use neural network model with 3 hidden layers. 1st layer with 50 nodes, second layer with 25 and third layer with 10. Using 100 epochs.
 This model resulted in Loss: 0.5522181987762451, Accuracy: 0.7308454513549805. Accuracy improved slightly and there was a super small decrease in loss.

**Run 3:** Use neural network model with 4 hidden layers distributed in the following way 50, 25, 10, and 5. Using 100 epochs.
This model resulted in Loss: 0.5533269643783569, Accuracy: 0.728863000869751. Decreased accuracy and increased loss.

**Run 4**: Rerun the hidden layers from Run 2 but with 200 epochs.
 This model resulted in Loss: 0.5548874139785767, Accuracy: 0.731195330619812. A negligible improvement in accuracy but increased loss.

The closest model was run 2/run 4 with either 100 or 200 epochs; it did not achieve the desired accuracy of 75% but got closest.

All optimization attempts are saved in "AlphabetSoupCharity_Optimization.h5”.

**Summary:**

The results of each of the model attempts did not improve the accuracy enough to reach the desired accuracy rate of 75%. In order to improve the model, it would be recommended to check for and remove outliers. Another consideration, that may effect results and accuracy, would be looking at changing the number of features.


In [29]:
# Export our model to HDF5 file
from google.colab import files

nn.save("AlphabetSoupCharity_Optimization.h5")
files.download("AlphabetSoupCharity_Optimization.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>